In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_metric_learning import losses, miners, distances

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_metric_learning import losses, miners, distances

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Dataset ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

# ---------------- CNN Encoder (LEARNED VERSION) ----------------
class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)          # (B, C, T)
        x = self.features(x).squeeze(-1)
        z = self.fc(x)
        return z                        # ❗ no normalize here

# ---------------- Full Model ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- Training ----------------
def train_and_eval(model, train_loader, test_loader,
                   metric_loss, miner,
                   epochs=500, patience=10):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)

    best_acc = -1
    patience_ctr = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            embeddings, logits = model(x)

            indices_tuple = miner(embeddings, y)
            loss = (
                metric_loss(embeddings, y, indices_tuple)
                + ce_loss(logits, y)
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # -------- Evaluation --------
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                preds = logits.argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print("Early stopping")
                break

    model.load_state_dict(best_state)
    return best_acc

# ---------------- Metric Losses (REFERENCE-STYLE) ----------------
encoder_losses = {
    "Triplet_L2": losses.TripletMarginLoss(
        margin=0.2,
        distance=distances.LpDistance(normalize_embeddings=True, p=2, power=2),
        smooth_loss=True
    ),
    "Triplet_Cosine": losses.TripletMarginLoss(
        margin=0.1,
        distance=distances.CosineSimilarity(),
        smooth_loss=True
    ),
    "NTXent": losses.NTXentLoss(temperature=0.2)
}

# ---------------- Miner (KEY FIX) ----------------
miner = miners.BatchEasyHardMiner(
    pos_strategy="semihard",
    neg_strategy="hard"
)

# ---------------- Experiment Loop ----------------
BASE = "models/Data/Data/62_classes/UserDependenet"
final_results = {}

for loss_name, metric_loss in encoder_losses.items():
    print(f"\n==============================")
    print(f" Encoder Loss: {loss_name}")
    print(f"==============================")

    accs = []

    for split in range(1, 11):
        print(f"\n----- SPLIT {split} -----")

        train_ds = EMGDataset(
            f"{BASE}/Train/X_train_{split}.npy",
            f"{BASE}/Train/y_train_{split}.npy"
        )
        test_ds = EMGDataset(
            f"{BASE}/Test/X_test_{split}.npy",
            f"{BASE}/Test/y_test_{split}.npy"
        )

        train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

        model = FullModel().to(device)

        acc = train_and_eval(
            model,
            train_loader,
            test_loader,
            metric_loss,
            miner,
            epochs=500,
            patience=10
        )

        print(f"Split Accuracy: {acc:.4f}")
        accs.append(acc)

        del model
        torch.cuda.empty_cache()

    final_results[loss_name] = np.mean(accs)
    print(f"\n>>> Average Accuracy ({loss_name}): {final_results[loss_name]:.4f}")

print("\n================ FINAL RESULTS ================")
for k, v in final_results.items():
    print(f"{k:20s} : {v:.4f}")

Using device: cuda

 Encoder Loss: Triplet_L2

----- SPLIT 1 -----
Epoch 001 | Val Acc: 0.0282
Epoch 002 | Val Acc: 0.2153
Epoch 003 | Val Acc: 0.3935
Epoch 004 | Val Acc: 0.5298
Epoch 005 | Val Acc: 0.6097
Epoch 006 | Val Acc: 0.6468
Epoch 007 | Val Acc: 0.6911
Epoch 008 | Val Acc: 0.7129
Epoch 009 | Val Acc: 0.6976
Epoch 010 | Val Acc: 0.7008
Epoch 011 | Val Acc: 0.7315
Epoch 012 | Val Acc: 0.7411
Epoch 013 | Val Acc: 0.7363
Epoch 014 | Val Acc: 0.7468
Epoch 015 | Val Acc: 0.7492
Epoch 016 | Val Acc: 0.7468
Epoch 017 | Val Acc: 0.7484
Epoch 018 | Val Acc: 0.7621
Epoch 019 | Val Acc: 0.7468
Epoch 020 | Val Acc: 0.7661
Epoch 021 | Val Acc: 0.7581
Epoch 022 | Val Acc: 0.7661
Epoch 023 | Val Acc: 0.7790
Epoch 024 | Val Acc: 0.7782
Epoch 025 | Val Acc: 0.7750
Epoch 026 | Val Acc: 0.7492
Epoch 027 | Val Acc: 0.7629
Epoch 028 | Val Acc: 0.7782
Epoch 029 | Val Acc: 0.7742
Epoch 030 | Val Acc: 0.7621
Epoch 031 | Val Acc: 0.7637
Epoch 032 | Val Acc: 0.7419
Epoch 033 | Val Acc: 0.7718
Early sto

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from pytorch_metric_learning import losses, miners, distances

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- DATASET ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)
        y = np.argmax(np.load(y_path), axis=1)

        # channel-wise normalization (CRITICAL)
        for i in range(X.shape[0]):
            for c in range(6):
                X[i, :, c] = (X[i, :, c] - X[i, :, c].mean()) / (X[i, :, c].std() + 1e-8)

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN + BiLSTM ENCODER ----------------
class CNNBiLSTMEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU()
        )

        self.bilstm = nn.LSTM(
            input_size=256,
            hidden_size=128,
            batch_first=True,
            bidirectional=True
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)      # (B, C, T)
        x = self.cnn(x)             # (B, 256, T')
        x = x.permute(0, 2, 1)      # (B, T', 256)

        out, _ = self.bilstm(x)
        feat = out[:, -1, :]        # last timestep

        z = self.fc(feat)
        return z

# ---------------- FULL MODEL ----------------
class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=62):
        super().__init__()
        self.encoder = CNNBiLSTMEncoder(emb_dim)
        self.classifier = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

# ---------------- TRAIN + EVAL ----------------
def train_and_eval(model, train_loader, test_loader,
                   metric_loss, miner,
                   epochs=300, patience=12):

    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=3e-4)

    best_acc = -1
    patience_ctr = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)

            # 🔥 CRITICAL FIX #1
            z = F.normalize(z, dim=1)

            # 🔥 CRITICAL FIX #2
            indices_tuple = miner(z, y)

            # 🔥 CRITICAL FIX #3 (λ = 0.1)
            loss = (
                ce_loss(logits, y)
                + 0.1 * metric_loss(z, y, indices_tuple)
            )

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        # -------- VALIDATION --------
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                preds = logits.argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:03d} | Val Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                print("Early stopping")
                break

    model.load_state_dict(best_state)
    return best_acc

encoder_losses = {
    "Triplet_L2": losses.TripletMarginLoss(
        margin=0.2,
        distance=distances.LpDistance(p=2),
        smooth_loss=True
    ),
    "Triplet_Cosine": losses.TripletMarginLoss(
        margin=0.1,
        distance=distances.CosineSimilarity(),
        smooth_loss=True
    ),
    "NTXent": losses.NTXentLoss(temperature=0.2)
}

miner = miners.BatchEasyHardMiner(
    pos_strategy="easy",
    neg_strategy="semihard"
)

BASE = "models/Data/Data/62_classes/UserDependenet"

final_results = {}

for loss_name, metric_loss in encoder_losses.items():
    print(f"\n========== {loss_name} ==========")
    accs = []

    for split in range(1, 20):
        train_ds = EMGDataset(
            f"{BASE}/Train/X_train_{split}.npy",
            f"{BASE}/Train/y_train_{split}.npy"
        )
        test_ds = EMGDataset(
            f"{BASE}/Test/X_test_{split}.npy",
            f"{BASE}/Test/y_test_{split}.npy"
        )

        train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=128)

        model = FullModel().to(device)

        acc = train_and_eval(
            model,
            train_loader,
            test_loader,
            metric_loss,
            miner
        )

        print(f"Split {split} Acc: {acc:.4f}")
        accs.append(acc)

        del model
        torch.cuda.empty_cache()

    final_results[loss_name] = np.mean(accs)

print("\n===== FINAL RESULTS =====")
for k, v in final_results.items():
    print(f"{k:20s}: {v:.4f}")

Using device: cuda

========== Triplet_L2 ==========
Epoch 001 | Val Acc: 0.0476
Epoch 002 | Val Acc: 0.0758
Epoch 003 | Val Acc: 0.1266
Epoch 004 | Val Acc: 0.1798
Epoch 005 | Val Acc: 0.2411
Epoch 006 | Val Acc: 0.3323
Epoch 007 | Val Acc: 0.3831
Epoch 008 | Val Acc: 0.3887
Epoch 009 | Val Acc: 0.4645
Epoch 010 | Val Acc: 0.5161
Epoch 011 | Val Acc: 0.4798
Epoch 012 | Val Acc: 0.5403
Epoch 013 | Val Acc: 0.5839
Epoch 014 | Val Acc: 0.5976
Epoch 015 | Val Acc: 0.6008
Epoch 016 | Val Acc: 0.6250
Epoch 017 | Val Acc: 0.6323
Epoch 018 | Val Acc: 0.6476
Epoch 019 | Val Acc: 0.6484
Epoch 020 | Val Acc: 0.6387
Epoch 021 | Val Acc: 0.6702
Epoch 022 | Val Acc: 0.6782
Epoch 023 | Val Acc: 0.6887
Epoch 024 | Val Acc: 0.6871
Epoch 025 | Val Acc: 0.6758
Epoch 026 | Val Acc: 0.6823
Epoch 027 | Val Acc: 0.6839
Epoch 028 | Val Acc: 0.7040
Epoch 029 | Val Acc: 0.6968
Epoch 030 | Val Acc: 0.7081
Epoch 031 | Val Acc: 0.7040
Epoch 032 | Val Acc: 0.7016
Epoch 033 | Val Acc: 0.6911
Epoch 034 | Val Acc: 0.

KeyboardInterrupt: 